In [6]:
import os
import numpy as np
from PIL import Image
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.svm import SVC
from sklearn.metrics import f1_score, accuracy_score
from torchvision import transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

# Define the paths
test_dir = 'C:\\Users\\taran\\Downloads\\SML\\CUB-200-2011-dataset-main\\CUB-200-2011-dataset-main\\test'
train_dir = 'C:\\Users\\taran\\Downloads\\SML\\CUB-200-2011-dataset-main\\CUB-200-2011-dataset-main\\new_train'
val_dir = 'C:\\Users\\taran\\Downloads\\SML\\CUB-200-2011-dataset-main\\CUB-200-2011-dataset-main\\new_val'

# Define transformations for image preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize image to fixed size
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize image
])

# Function to extract features using PCA
def extract_features(dataset_dir):
    features = []
    labels = []
    for class_folder in tqdm(sorted(os.listdir(dataset_dir))):
        class_path = os.path.join(dataset_dir, class_folder)
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            image = Image.open(img_path)
            image = transform(image).numpy().flatten()  # Flatten image tensor to 1D array
            features.append(image)
            labels.append(int(class_folder[:3]) - 1)  # Extract class index from folder name
    features = np.array(features)
    # reduced_features = pca_model.transform(features)
    return features, labels



# Load the dataset
train_features, train_labels = extract_features(train_dir)
val_features, val_labels = extract_features(val_dir)
test_features, test_labels = extract_features(test_dir)




100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [01:14<00:00,  2.70it/s]


In [9]:
# Train PCA
n_components = 1000  # Number of components to keep
pca = PCA(n_components=n_components)
pca.fit(train_features)

# Apply PCA transformation
train_features_pca = pca.transform(train_features)
val_features_pca = pca.transform(val_features)
test_features_pca = pca.transform(test_features)



In [10]:
# Train QDA
qda = QDA()
qda.fit(train_features_pca, train_labels)

# Train SVM
svm = SVC()
svm.fit(train_features_pca, train_labels)

# Evaluate on test dataset
qda_predictions = qda.predict(test_features_pca)
svm_predictions = svm.predict(test_features_pca)

qda_f1 = f1_score(test_labels, qda_predictions, average='weighted')
svm_f1 = f1_score(test_labels, svm_predictions, average='weighted')

qda_accuracy = accuracy_score(test_labels, qda_predictions)
svm_accuracy = accuracy_score(test_labels, svm_predictions)

print(f"QDA - F1 Score: {qda_f1}, Accuracy: {qda_accuracy}")
print(f"SVM - F1 Score: {svm_f1}, Accuracy: {svm_accuracy}")

C:\Users\taran\anaconda3\envs\NewGPU\lib\site-packages\sklearn\discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


QDA - F1 Score: 0.005926068045658185, Accuracy: 0.005695547117707974
SVM - F1 Score: 0.021497309886194402, Accuracy: 0.030031066620642043


In [11]:
svm = SVC()
svm.fit(train_features, train_labels)

SVC()

In [ ]:
svm_predictions = svm.predict(test_features)

In [ ]:
svm_f1 = f1_score(test_labels, svm_predictions, average='weighted')
svm_accuracy = accuracy_score(test_labels, svm_predictions)
print(f"SVM - F1 Score: {svm_f1}, Accuracy: {svm_accuracy}")